In [11]:
import os
import keras
import keras_nlp
import numpy as np
import PIL
import requests
import io
import matplotlib
import re
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image, ImageFilter
import io
import requests
import pandas as pd

In [12]:
os.environ["KERAS_BACKEND"] = "jax"
keras.config.set_floatx("bfloat16")

In [13]:
paligemma = keras_nlp.models.PaliGemmaCausalLM.from_preset("pali_gemma_3b_mix_224")
paligemma.summary()

Preprocessor: "pali_gemma_causal_lm_preprocessor_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ pali_gemma_tokenizer (PaliGemmaTokenizer)          │                                             257,152 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "pali_gemma_causal_lm_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ images (InputLayer)           │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ response_mask (InputLayer)    │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pali_gemma_backbone           │ (None, None, 2048)        │   2,923,335,408 │ images[0][0],              │
│ (PaliGemmaBackbone)           │                           │                 │ padding_mask[0][0],        │
│                               │                           │                 │ response_mask[0][0],       │
│                               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 257152)      │     526,647,296 │ pali_gemma_backbone[0][0]  │
│ (ReversibleEmbedding)         │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ get_item_1 (GetItem)          │ (None, None, 257152)      │               0 │ token_embedding[1][0]      │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,923,335,408 (5.45 GB)

 Trainable params: 2,923,335,408 (5.45 GB)

 Non-trainable params: 0 (0.00 B)

In [14]:
def sharpen_image(image):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])  
    sharpened = cv2.filter2D(np.array(image), -1, kernel)
    return Image.fromarray(sharpened)

In [15]:
def crop_and_resize(image, target_size):
    return image.resize(target_size, PIL.Image.Resampling.LANCZOS)

def read_image(url, target_size):
    image = PIL.Image.open(requests.get(url, stream=True).raw).convert('RGB')
    image = crop_and_resize(image, target_size)
    image = np.array(image)
    return image


In [16]:
target_size = (224, 224)
image_url = 'https://m.media-amazon.com/images/I/81N73b5khVL.jpg'
item_image = read_image(image_url, target_size)


In [17]:
test_df = pd.read_csv("/kaggle/input/amazonml/test.csv")
sample = test_df
result = pd.DataFrame({"index": [], "prediction": []})

In [ ]:
for (idx, (index, url, id, ent_type)) in enumerate(sample.values):
    print(f"{idx}: {url}")
    item_image = read_image(url, target_size)
    
    if ent_type == "item_weight" or ent_type == "maximum_weight_recommendation":
        ent_type = "net weight"
    
    prompt = f'What is the item {ent_type}? Mention the unit too\n'
    output = paligemma.generate(
        inputs={
            "images": item_image,
            "prompts": prompt,
        }
    )
    print(output)
    result = pd.concat([result, pd.DataFrame({"index": [idx], "prediction": [output.split("\n")[1]]})], ignore_index=True)
    
    if idx == 5000:
        break
 

0: https://m.media-amazon.com/images/I/110EibNyclL.jpg
What is the item height? Mention the unit too
100cm - 103.5cm
1: https://m.media-amazon.com/images/I/11TU2clswzL.jpg
What is the item width? Mention the unit too
158
2: https://m.media-amazon.com/images/I/11TU2clswzL.jpg
What is the item height? Mention the unit too
16cm
3: https://m.media-amazon.com/images/I/11TU2clswzL.jpg
What is the item depth? Mention the unit too
158
4: https://m.media-amazon.com/images/I/11gHj8dhhrL.jpg
What is the item depth? Mention the unit too
16
5: https://m.media-amazon.com/images/I/11gHj8dhhrL.jpg
What is the item height? Mention the unit too
110cm/43.3"
6: https://m.media-amazon.com/images/I/11gHj8dhhrL.jpg
What is the item width? Mention the unit too
10cm/3.93"
7: https://m.media-amazon.com/images/I/11lshEUmCrL.jpg
What is the item height? Mention the unit too
1.4 in 3.5 cm
8: https://m.media-amazon.com/images/I/21+i52HRW4L.jpg
What is the item width? Mention the unit too
30 cm
9: https://m.media-am

In [ ]:
result.to_csv("result.csv")